# Sensorless Drive Diagnosis Project

## Purpose of the project

<b>To build a multi-class classification model with numerical attributes</b>


The task is to classify the condition of a synchron motor by using current measurements in the motor. 

<b>Research Paper </b>

[The original paper](https://www.researchgate.net/publication/262698433_Sensorlose_Zustandsuberwachung_an_Synchronmotoren)  is in German language. [Here](https://www.researchgate.net/publication/264273485_Feature_Extraction_and_Reduction_Applied_to_Sensorless_Drive_Diagnosis) is the English version. 

After the advances in industrial information technology, condition monitoring methods are becoming increasingly important. The phase currents are used at the evaluation of the process data without additional, cost-intensive sensors and the determination the damage status of a syn-characterize chronomotors and the connected components. 


## About the dataset

<b>Dataset used in the analysis: </b> Sensorless Drive Diagnosis dataset (SDD): 

https://archive.ics.uci.edu/ml/datasets/Dataset+for+Sensorless+Drive+Diagnosis 

The dataset contains information collected from electric current drive signals of a synchronous electric motor. The current signals are measured with a current probe and an oscilloscope on two phases.

<b>Raw Data: </b> 

In the raw data there are a lot of measurements (different speeds, load moments and load forces) and different health conditions (motor ok, failure in the bearings, etc. ) 

The raw data is timeseries data. However the dataset provided are only the extracted features from the raw data. 

<b>Features and classes: </b>

There are 48 continous predictive features. The target feature contains 11 classes. 


## Setup

Python 3.7.4
sklearn.__version__ 0.23.2
pandas.__version__ 0.25.1
numpy.__version__ 1.17.2
matplotlib.__version__ 3.1.1
seaborn.__version__ 0.9.0
xgboost.__version__ 0.90

In [51]:
print(sklearn.__version__)




NameError: name 'sklearn' is not defined

### Importing Libraries

In [ ]:
import time

import pandas as pd
import numpy as np
np.random.seed(42)

# plot pretty figures
import matplotlib.pyplot as plt
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
import seaborn as sns
sns.set_style('darkgrid', {'axes.facecolor': '0.9'})

# sklearn machine learning 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, f1_score, fbeta_score, r2_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

# sklearn multi layer perceptron
from sklearn.neural_network import MLPClassifier

import xgboost as xgb

### Functions 

In [ ]:
# functions are in a Python script at the same folder. 
%run -i 'functions.py'

In [ ]:
# This cell is the same as the above cell. No need to run this if you downloaded the GitHub repo. 

def plot_coefficients(clf):
    """
        The function to plot the coefficients of logistic regression model.
  
        Parameters:
            clf: classifier model   
    """
    
    weights_clf = pd.Series(clf.coef_[0], index=scaled_X_train_log.columns.values)
    weights_clf.sort_values(inplace=True)
    plt.figure(figsize=(20, 6))
    plt.xticks(rotation=90)
    #barplot
    features = plt.bar(weights_clf.index, weights_clf.values)
    
    
    
def model_evaluation(X_train, X_test, y_train, y_test, y_pred_train, y_pred_test):
    """
        The function to print the metrics of the model evaluation.
        Printed metrics: Accuracy scores, 
                         confusion matrixes, 
                         and classification reports for train and test sets, 
  
        Parameters: Train, test, and prediction values
                    X_train, X_test, 
                    y_train, y_test, 
                    y_pred_train, y_pred_test   
    """

    print('MODEL EVALUATION METRICS:\n',
          '-----------------------------------------------------')
    print('Train Set Accuracy Score', round(accuracy_score(y_train, y_pred_train), 6))
    print('Test Set Accuracy Score', round(accuracy_score(y_test, y_pred_test),6))
    print('-----------------------------------------------------\n')

    print('Confusion Matrix for train & test set: \n',
          '\nTrain set\n',
          confusion_matrix(y_train, y_pred_train), '\n'
          '\n\nTest set\n',)
    print(confusion_matrix(y_test, y_pred_test), '\n')


    print('-----------------------------------------------------')
    print('\nClassification Report for train & test set\n',
          '\nTrain set\n',
          classification_report(y_train, y_pred_train),
          '\n\nTest set\n',
          classification_report(y_test, y_pred_test))

    print('-----------------------------------------------------\n')

    # print('roc auc score for train and test set:\n ',
    #       round(roc_auc_score(y_train, y_pred_train), 4),
    #       round(roc_auc_score(y_test, y_pred_test), 4))
    
    
def plot_feature_importances(model):
    """
        The function to plot the coefficients of tree based models.
  
        Parameters:
            model: classifier model  
    """
    n_features = X_train.shape[1]
    plt.figure(figsize=(20, 15))
    #barplot
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), X_train.columns.values)
    plt.title('Comparison of Feature Importances')
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')
    

## Load Dataset

In [ ]:
df = pd.read_csv("sensorless_drive_diagnosis.txt", delim_whitespace=True, header=None)
header_names = ['feat' + str(i) for i in range(1, df.shape[1])]
header_names.append('class')
df.set_axis(header_names, axis=1, inplace=True)
display(df.head())
display(df.tail())
display(df.describe())

## Basic EDA

### Checking Missing Values

There are not any misssing values. 

In [ ]:
df.isna().sum().sum()

### Checking the basic info

48non-null float(features), 1 non-null integer(target)

In [ ]:
df.info()

### Checking Class Sizes

There is not a class imbalance issue . All the classes have the same number of samples. Because there is not a class imbalance problem, accuracy is enough as a metric. 

We are going to check <b>precision, recall, and F1 scores </b>, but in the grid search and random search we are going to use accuracy to evaluate model performance. 

We are not going to calculate <b>F-Beta score</b> and <b>Cohen's Kappa score </b>  in this project. 

In [ ]:
df.groupby('class').size()

### Seperating Target and Features

In [ ]:
X = df.drop(['class'], axis=1)
y = df['class']

### Making a random sample 


In [ ]:
df_sample = df.sample(500)

## Data Visualizations

### Histograms
Results of the histograms: There are a lot of features that have exact same distribution. There can be a high correlation between features. For example <b>feat24</b>, <b>feat25</b>, <b>feat26</b>, <b>feat27</b> are the same. 

A heatmap can help us see the amount of correlation. 

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 16
fig_size[1] = 48
plt.rcParams["figure.figsize"] = fig_size
# plt.figure(figsize=(16,48))
X.hist(layout=(12,4))
plt.show()


### Heatmaps

Heatmap with annotations show the correlated columns and the amount of correlation. 

#### Heatmap 1
At the below map, <b>18 columns</b> are 100% correlated to each other. Dropping them in the beginning of the analysis. 


In [ ]:
df_heatmap = df_sample.drop(['class'], axis=1)
plt.figure(figsize=(40, 40))
ax = sns.heatmap(df_heatmap.corr(), center=0, linewidths=.5, annot=True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

#### Dropping Correlated Columns

18 columns are dropped, 30 columns are kept

In [ ]:
# 100% correlated columns are dropped
columns_to_drop = ['feat8', 'feat9', 'feat11', 'feat12', 'feat16',
                   'feat20','feat21','feat22','feat23','feat24', 
                   'feat32','feat33','feat35', 'feat36',
                   'feat44','feat45','feat47','feat48']
df = df.drop(columns_to_drop, axis=1)

#### Heatmap 2

According to the heatmap below, there are not 100% correlated columns but <b>feat13</b>, <b>feat15</b>, <b>feat17</b> are also highly correlated to <b>feat14</b>. They can affect the logistic regression model. 

In [ ]:
df_heatmap = df.drop(['class'], axis=1) # dropping target columns and making correlation heatmap of features. 
plt.figure(figsize=(40, 40)) 
ax = sns.heatmap(df_heatmap.corr(), center=0, linewidths=.5, annot=True, annot_kws={"size": 20})
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5) # this line is to fix a bug at the heatmap
plt.show()

#### Boxplots

##### Seperating target and features after dropping columns

In [ ]:
X = df.drop(['class'], axis=1)
y = df['class']

##### Scaling Features To Get Better Boxplots (MinMax Scaler)

In [ ]:
scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X)
# scaled_X = pd.DataFrame(scaled_x_train, columns=X_train.columns)

In [ ]:
sns.set(rc={'figure.figsize': (35, 20)},font_scale=2.1)  # Set font scale   
g = sns.boxplot(data=scaled_X)
for item in g.get_xticklabels():  # Rotate x labels to 70 degrees angle
    item.set_rotation(70)
plt.title('Boxplots Of Features');

In [ ]:
selected_features = ['feat1', 'feat2', 'feat3', 'feat4', 'feat5', 
                     'feat6', 'feat7', 'feat10', 'feat13', 'feat14', 
                     'feat15', 'feat17', 'feat18', 'feat19', 'feat25']
    
fig, ax = plt.subplots(5, 3)
for variable, subplot in zip(selected_features, ax.flatten()):
    sns.lineplot(df.index, df[variable], ax=subplot, hue=df['class'])

In [ ]:
selected_features = ['feat26', 'feat27', 'feat28', 'feat29', 'feat30', 
                     'feat31', 'feat34', 'feat37', 'feat38', 'feat39', 
                     'feat40', 'feat41', 'feat42', 'feat43', 'feat46']
    
fig, ax = plt.subplots(5, 3)
for variable, subplot in zip(selected_features, ax.flatten()):
    sns.lineplot(df.index, df[variable], ax=subplot, hue=df['class'])


## Modelling 

### Models in this notebook: Logistic Regression, Random Forest Classifier, XGBoost Classifier, Multi Layer Perceptron

I decided to use logistic regression as a baseline model. 

I chose random forest and XGBoost as main models. Because they are good at handling complex, non-linear relationships.

I added a multiclass perceptron as a deep learning model. 


### Splitting The Data Into Test and Train Sets

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

### Scaling Training Data for Logistic Regression Model (Standard Scaler)

In [53]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)
scaled_X_train = pd.DataFrame(scaled_X_train, columns=X_train.columns)
scaled_X_test = pd.DataFrame(scaled_X_test, columns=X_test.columns)

### Baseline Model: Logistic Regression

In [54]:
scaled_X_train_log = scaled_X_train.drop(['feat13', 'feat15', 'feat17' ], axis=1)
scaled_X_test_log = scaled_X_test.drop(['feat13', 'feat15', 'feat17'], axis=1)

In [55]:
scaled_X_test_log.shape

(11702, 27)

In [ ]:
log = LogisticRegression(solver='saga', max_iter=5000)
log.fit(scaled_X_train_log, y_train)
y_pred_train = log.predict(scaled_X_train_log)
y_pred_test = log.predict(scaled_X_test_log)

#### Model Evaluation Metrics

In [ ]:
model_evaluation(scaled_X_train_log, scaled_X_test_log, y_train, y_test, y_pred_train, y_pred_test)

In [ ]:
plot_coefficients(log)

#### Results of Logistic Regression

##### Assumptions of Logistic Regression

Logitic Regression has normality, linearity, and homoscedasticity assumptions. We can make log normalization to have a more normal distribution. We can check the residuals for homoscedasticty. 

##### Feature Selection

We should get rid of outliers that have too much influence and leverage. We should eliminate the correlated columns by Variance Inflation Factor(VIF) or Predictive Power Score(PPS). We can also use Lasso and Ridge penalties for feature selection. We can apply other filter or wrapper methods. 

Because this is my vanilla model, I decided to continue with other models. 

### Random Forest Classification Model

In [ ]:
rf_clf = RandomForestClassifier(n_jobs=4, class_weight='balanced', random_state=42, n_estimators= 80)
rf_clf.fit(X_train, y_train)
y_pred_train = rf_clf.predict(X_train)
y_pred_test = rf_clf.predict(X_test)

In [ ]:
model_evaluation(X_train, X_test, y_train, y_test, y_pred_train, y_pred_test)

In [ ]:
plot_feature_importances(rf_clf)

### Pipeline and Grid Search - Random Forest Classification Model

In [ ]:
# pipe = Pipeline([('classifier', RandomForestClassifier(random_state=123))])
# grid = [{'classifier__criterion': ['gini', 'entropy'],
#          'classifier__n_estimators':[100, 120, 150],
#          'classifier__max_depth': [12, 14],
#          'classifier__min_samples_split': [4, 6, 8]}]
# clf = GridSearchCV(estimator=pipe, param_grid=grid,
#                    cv=5, n_jobs=-1)


# print("Grid search..")
# search_time_start = time.time()

# clf.fit(X_train, y_train)
# y_pred_train = clf.predict(X_train)
# y_pred_test = clf.predict(X_test)

# search_time_end = time.time()
# print('Time for grid search in seconds', search_time_end - search_time_start )

In [ ]:
print("Best Parameter Combination Found During Grid Search:\n ", clf.best_params_)

#### Grid Search Results 

Below are the best parameters of above grid search. I commented out the cell, it takes 22 minutes time to run the grid search on my MacBook Pro 
       
       
  
  Processor 2.2 GHz Intel Core i7,     Memory: 16 GB 1600 MHz DDR3     Operation System: macOS Mojave. 
  
 You can just uncomment and run the above cell if needed. 
 



      'classifier__criterion': 'entropy'
      'classifier__max_depth': 14
      'classifier__min_samples_split': 4
      'classifier__n_estimators': 150

In [ ]:
rf_clf = RandomForestClassifier(n_jobs=4, class_weight='balanced', random_state=42, n_estimators= 150,
                             criterion = 'entropy', max_depth = 14, min_samples_split = 4)
rf_clf.fit(X_train, y_train)
y_pred_train = rf_clf.predict(X_train)
y_pred_test = rf_clf.predict(X_test)

In [ ]:
model_evaluation(X_train, X_test, y_train, y_test, y_pred_train, y_pred_test)

In [ ]:
plot_feature_importances(rf_clf)

### Random Search - XGBoost Classifier 

I ran a lot of different models and grid searches before starting this random search. I didn't include them all. We can find the best combination after many iterations depending on the computational power. 

In [ ]:
clf = xgb.XGBClassifier(objective="multi:softmax")
param_grid = {
        'silent': [False],
        'max_depth': [6, 10, 15, 20],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}
RS_clf = RandomizedSearchCV(estimator=clf, param_distributions=param_grid,
                            cv=5, n_jobs=-1)
print("Randomized search..")
search_time_start = time.time()

RS_clf.fit(X_train, y_train)
y_pred_train = RS_clf.predict(X_train)
y_pred_test = RS_clf.predict(X_test)

search_time_end = time.time()
print('Time for randomized search in minutes: ', round((search_time_end - search_time_start)/60 ) )


In [ ]:
print("Best Parameter Combination Found During Random Search:\n ", RS_clf.best_params_)

Below are the best parameters of above random search. I commented out the cell, it takes 38 minutes to run the grid search on my MacBook Pro 
       
       
  
  Processor 2.2 GHz Intel Core i7,     Memory: 16 GB 1600 MHz DDR3     Operation System: macOS Mojave. 
  
 You can just uncomment and run the above cell if needed. 
 



    'subsample': 0.9, 

    'reg_lambda': 0.1, 

    'n_estimators': 100, 

    'min_child_weight': 1.0, 

    'max_depth': 10, 

    'learning_rate': 0.1, 

    'gamma': 0.5, 

    'colsample_bytree': 0.7

### Final XGBoost Model 

In [ ]:
clf = xgb.XGBClassifier(objective='multi:softmax', nthread=4, scale_pos_weight=3,
                        colsample_bytree= 0.7, gamma= 0.5, learning_rate= 0.1, 
                        max_depth= 10,min_child_weight= 1.0, reg_lambda= 1.0, 
                        silent= False, subsample= 0.9, seed=42, n_estimators=100)
search_time_start = time.time()
clf.fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

search_time_end = time.time()
print('Time for the final XG Boost model in seconds: ', round(search_time_end - search_time_start))


#### Model Evaluation Metrics

In [ ]:
model_evaluation(X_train, X_test, y_train, y_test, y_pred_train, y_pred_test)

In [ ]:
plot_feature_importances(clf)

### Multi Layer Perceptron Model

In [ ]:
search_time_start = time.time()

print('Multi Layer Perceptron')

mlp = MLPClassifier(hidden_layer_sizes=(128,128,128,128,128,128), activation='relu', 
                    solver='adam', max_iter=200, verbose=True)
mlp.fit(X_train,y_train)
Y_train_mlp_pred_df = mlp.predict(X_train)
Y_valid_mlp_pred_df = mlp.predict(X_test)

search_time_end = time.time()
print('Time for the final MLP model in seconds: ', search_time_end - search_time_start )


#### MLP Model Results

In [ ]:
cm_mlp_train = confusion_matrix(y_train, Y_train_mlp_pred_df)
print(cm_mlp_train)
mlp_train_accuracy = cm_mlp_train.trace()/cm_mlp_train.sum()

print('MLP accuracy on a train set is: ', round(mlp_train_accuracy, 6))
print(60*'-')

cm_mlp = confusion_matrix(y_test, Y_valid_mlp_pred_df)
print(cm_mlp)
mlp_test_accuracy = cm_mlp.trace()/cm_mlp.sum()

print('MLP accuracy on a test set is: ', round(mlp_test_accuracy, 6))


In [ ]:
def baseline_model(input_dim,output_dim):
    model = Sequential()
    model.add(Dense(input_dim, input_dim=input_dim, activation='relu'))
    model.add(Dense(input_dim*2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(input_dim*2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(input_dim*2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(input_dim*2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(input_dim//2, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model